Needed:

* **Queue** = "solo/duo"

  - queue_type: "Ranked Solo/Duo"

   -  highestTierAchieved: RANKED_SOLO_5x5

* ranked match
* several players
* different players' ranks --> **tier**


In [1]:
import requests
import time
import random
import json
from google.colab import files

In [2]:
def get_leagues(platform, api):
  """
  returns 200 summonerNames from master, grandmaster and challenger leagues in the given platform
  """

  leagues = ['challengerleagues', 'grandmasterleagues', 'masterleagues']
  
  url_challenger = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[0]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_challenger)
  lis = response.json()['entries']
  summonerName_c = [sub['summonerName'] for sub in lis]

  url_grandmaster = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[1]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_grandmaster)
  lis = response.json()['entries']
  summonerName_g = [sub['summonerName'] for sub in lis]

  url_master = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[2]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_master)
  lis = response.json()['entries']
  summonerName_m = [sub['summonerName'] for sub in lis]

  summonerName = summonerName_c + summonerName_g + summonerName_m

  return summonerName



def get_puuid(platform, api, summonerNames):

  puuid_list = []
  for name in summonerNames:

    url = 'https://' + platform.lower() + '.api.riotgames.com/lol/summoner/v4/summoners/by-name/'  + name + '?api_key=' + api
    response = requests.get(url).json()
    if response.get('status'):
      continue
    puuid = response['puuid']
    puuid_list.append(puuid)
    print(len(puuid_list))
  return puuid_list



def get_matchId(region, api, puuids):

  matchId_list = []
  for puuid in puuids:
    url = 'https://'+ region +'.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid +'/ids?start=0&count=10&api_key=' + api
    response = requests.get(url).json()

    # If rate limit is exceeded
    if isinstance(response, dict):
      print(response['status']['message'])
      time.sleep(120)
      url = 'https://'+ region +'.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid +'/ids?start=0&count=10&api_key=' + api
      response = requests.get(url).json()

    matchId_list.extend(response)

  return matchId_list



def get_matches(region, api, matchIds):

  matches = []
  i = 0
  random_indices = random.sample(range(0, len(matchIds)-1), 1000)
  for ind in random_indices:
    time.sleep(0.01)
    url = 'https://' + region + '.api.riotgames.com/lol/match/v5/matches/' + matchIds[ind] + '?api_key=' + api
    response = requests.get(url).json()

    # If rate limit is exceeded
    while response.get('status'):
      print(response['status']['message'])
      time.sleep(40)
      url = 'https://' + region + '.api.riotgames.com/lol/match/v5/matches/' + matchIds[ind] + '?api_key=' + api
      response = requests.get(url).json()
    
    i+=1
    print(i)
    matches.append(response)

  return matches

In [3]:
api = '' #add your api here

platform = ['BR1', 'eun1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'RU', 'TR1']
region = 'americas'

In [4]:
summonerNames = get_leagues(platform[0], api)

In [5]:
len(summonerNames)

6863

In [ ]:
res = get_puuid(platform[0], api, summonerNames)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [ ]:
len(res)

In [ ]:
res

In [ ]:
match_ids = get_matchId(region, api, res)

In [ ]:
match_ids

In [ ]:
len(match_ids)

In [ ]:
matches = get_matches(region, api, match_ids)

In [ ]:
matches

In [ ]:
len(matches)

In [ ]:
with open('matches', 'w') as f:
  json.dump(matches, f)

In [ ]:
files.download("matches1000")